In [1]:
# @title 1.1 Install Condacolab (< 1min)
%%time

! pip install -q condacolab
import condacolab
condacolab.install()

#@markdown *Kernel will restart automatically after this installation*
#@markdown
#@markdown *Keep connected to the same runtime and proceed to the next code block*


⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:15
🔁 Restarting kernel...
CPU times: user 733 ms, sys: 228 ms, total: 961 ms
Wall time: 25.2 s


In [1]:
# @title 1.2 Install Packages and Data (~ 5min)
%%time

# Install Reduce2 (cctbx-base)
! conda install cctbx-base


# Install Prody and py3Dmol
! pip install prody py3Dmol


# Download Phenix Project geostd (restraint) Library
goestd_repo = "https://github.com/phenix-project/geostd.git"
! git clone {goestd_repo}


# Install Vina, Meeko (develop branch) and Dependencies
! conda install numpy scipy rdkit vina
! git clone --single-branch --branch develop https://github.com/forlilab/Meeko.git
! cd Meeko; pip install --use-pep517 -e .; cd ..


# Install Scrubber (develop branch)
! git clone --single-branch --branch develop https://github.com/forlilab/scrubber.git
! cd scrubber; pip install --use-pep517 -e .; cd ..


# Download Vina Executables
! wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64
! mv vina_1.2.5_linux_x86_64 vina; chmod +x vina;
! wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_split_1.2.5_linux_x86_64
! mv vina_split_1.2.5_linux_x86_64 vina_split; chmod +x vina_split

Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cctbx-base


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    biopython-1.84             |  py310hc51659f_0         2.7 MB  conda-forge
    brotli-1.1.0               |       hd590300_1          19 KB  conda-forge
    brotli-bin-1.1.0           |       hd590300_1          19 KB  conda-forge
    ca-certificates-2024.8.30  |       hbcca054_0         155 KB  conda-forge
    cctbx-base-2024.8          |  py310h159de0f_0        61.0 MB  conda-forge
    certifi-2024.8.30          |     pyhd8ed1ab_0         160 KB  conda-forge
    contou

In [2]:
# @title 1.3 Import Modules & Helper Functions (< 1s)
%%time

# Import modules
import sys, platform
from prody import *
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem
import rdkit, py3Dmol
print("rdkit version:", rdkit.__version__)
print("py3Dmol version:", py3Dmol.__version__)
from ipywidgets import interact, IntSlider
import ipywidgets, copy


# Helper functions
def locate_file(from_path = None, query_path = None, query_name = "query file"):

    if not from_path or not query_path:
        raise ValueError("Must specify from_path and query_path")

    possible_path = list(from_path.glob(query_path))

    if not possible_path:
        raise FileNotFoundError(f"Cannot find {query_name} from {from_path} by {query_path}")

    return_which = (
        f"using {query_name} at:\n"
        f"{possible_path[0]}\n"
    )
    print(return_which)

    return possible_path[0]


def confgen_from_smiles(smi = None, outputSDF = "ligand.sdf", numConfs = 50, maxIters = 5000):

    def which_conformer(opt_res):

      normal_res = [x for x in opt_res if x[0]==0]

      if len(normal_res)==0: # all opt failed
          return -1
      else:
          min_eng = min([x[1] for x in normal_res])
          for conf_id,r in enumerate(opt_res):
            if (r[0]==0 and r[1]==min_eng):
                return conf_id

    if smi is None:
      raise ValueError("Must specify smi")

    m = Chem.MolFromSmiles(smi)

    mh = Chem.AddHs(m)

    params = AllChem.ETKDGv3()
    params.numThreads = 0

    cids = AllChem.EmbedMultipleConfs(mh, numConfs, params)
    res = AllChem.MMFFOptimizeMoleculeConfs(mh, numThreads = 0, maxIters = maxIters)

    minconf = which_conformer(res)
    if minconf!=-1:
        with Chem.SDWriter(outputSDF) as w:
          w.write(mh, confId=minconf)
    else:
      raise RuntimeError("Optimization didn't converge. No conformer/SDF will be written")

# Commandline scripts
scrub = locate_file(from_path = Path("/usr/local/bin"), query_path = "scrub.py", query_name = "scrub.py")
mk_prepare_ligand = locate_file(from_path = Path("/usr/local/bin"), query_path = "mk_prepare_ligand.py", query_name = "mk_prepare_ligand.py")
mk_prepare_receptor = locate_file(from_path = Path("/usr/local/bin"), query_path = "mk_prepare_receptor.py", query_name = "mk_prepare_receptor.py")
mk_export = locate_file(from_path = Path("/usr/local/bin"), query_path = "mk_export.py", query_name = "mk_export.py")


# Locate reduce2 in conda install prefix
full_py_version = platform.python_version()
major_and_minor = ".".join(full_py_version.split(".")[:2])
env_path = Path("/usr/local") # default conda install prefix on Colab
reduce2_path = f"lib/python{major_and_minor}/site-packages/mmtbx/command_line/reduce2.py"
reduce2 = locate_file(from_path = env_path, query_path = reduce2_path, query_name = "reduce2.py")


# Locate geostd in current path
geostd_path = locate_file(from_path = Path.cwd(), query_path = "geostd", query_name = "geostd")

rdkit version: 2023.09.6
py3Dmol version: 2.4.0
using scrub.py at:
/usr/local/bin/scrub.py

using mk_prepare_ligand.py at:
/usr/local/bin/mk_prepare_ligand.py

using mk_prepare_receptor.py at:
/usr/local/bin/mk_prepare_receptor.py

using mk_export.py at:
/usr/local/bin/mk_export.py

using reduce2.py at:
/usr/local/lib/python3.10/site-packages/mmtbx/command_line/reduce2.py

using geostd at:
/content/geostd

CPU times: user 904 ms, sys: 114 ms, total: 1.02 s
Wall time: 1.08 s


In [3]:
# @title # 2.1 [Basic Docking] Ligand Preparation (< 5s)
%%time

#@markdown ## (1) Conformer Generation with Scrubber

# @markdown *Input*
# Specify ligand Smiles
ligand_Smiles = "CNc1ncc(cn1)CN2CCC[C@@H](C2)c3nc(cc(n3)O)c4cccs4" #@param {type:"string"}
# @markdown *Options*
pH = 8 #@param {type:"raw"}
args = ""
skip_tautomer = True #@param {type:"boolean"}
if skip_tautomer:
  args += "--skip_tautomer "
skip_acidbase = False #@param {type:"boolean"}
if skip_acidbase:
  args += "--skip_acidbase "

# @markdown *Output*
ligandSDF = "51B_scrubbed.sdf" #@param {type:"string"}


# Write scrubbed protomer(s) and conformer(s) to SDF
! python {scrub} "{ligand_Smiles}" -o {ligandSDF} --ph {pH} {args}


# Visualization with py3Dmol
view = py3Dmol.view()
view.addModel(open(ligandSDF, 'r').read(),'sdf')
view.zoomTo()
view.setBackgroundColor('white')
view.addStyle({'stick': {'colorscheme':'yellowCarbon'}})
view.show()


#@markdown ---
#@markdown ## (2) Ligand Preparation with Meeko

# Prepare ligand PDBQT
# @markdown *Output*
ligandPDBQT = "51B.pdbqt" #@param {type:"string"}
! python {mk_prepare_ligand} -i {ligandSDF} -o {ligandPDBQT}

Scrub completed.
Summary of what happened:
Input molecules supplied: 1
mols processed: 1, skipped by rdkit: 0, failed: 0
nr isomers (tautomers and acid/base conjugates): 1 (avg. 1.000 per mol)
nr conformers:  1 (avg. 1.000 per isomer, 1.000 per mol)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

CPU times: user 38.3 ms, sys: 6.96 ms, total: 45.2 ms
Wall time: 2.93 s


In [5]:
# @title # 2.2 [Basic Docking] Receptor Preparation (< 30s)
%%time

#@markdown ## (1) Add Hydrogens to Receptor with Reduce2

# @markdown *Input*
# Download PDB file
pdb_token = "5c45" #@param {type:"string"}
! curl "http://files.rcsb.org/view/{pdb_token}.pdb" -o "{pdb_token}.pdb"


# Export receptor atoms
atoms_from_pdb = parsePDB(pdb_token)
# @markdown *ProDy Options*
receptor_selection = "not water and not hetero" #@param {type:"string"}
receptor_atoms = atoms_from_pdb.select(receptor_selection)
# @markdown *Prody Output*
prody_receptorPDB = "5c45_receptor_atoms.pdb" #@param {type:"string"}
writePDB(prody_receptorPDB, receptor_atoms)


# Add CRYST1 card (temporarily required for reduce2)
# @markdown *Reduce Input*
reduce_inputPDB = "5c45_receptor.pdb" #@param {type:"string"}
! cat <(grep "CRYST1" "{pdb_token}.pdb") {prody_receptorPDB} > {reduce_inputPDB}


# Run reduce2
# @markdown *Reduce Options*
reduce_opts = "approach=add add_flip_movers=True" #@param {type:"string"}
! export MMTBX_CCP4_MONOMER_LIB="{geostd_path}"; python {reduce2} {reduce_inputPDB} {reduce_opts}
# Default name of reduce output...
prepare_inPDB = "5c45_receptorFH.pdb" #@param {type:"string"}

#@markdown ---
#@markdown ## (2) Receptor Preparation with Meeko

# Specify Box
# Center at ligand
atoms_from_pdb = parsePDB(pdb_token)
# @markdown *Center the box at...*
ligand_selection = "resname 51B" #@param {type:"string"}
ligand_atoms = atoms_from_pdb.select(ligand_selection)
center_x, center_y, center_z = calcCenter(ligand_atoms)


# @markdown *For future reference, export ligand's original position to...*
prody_ligandPDB = "LIG.pdb" #@param {type:"string"}
writePDB(prody_ligandPDB, ligand_atoms)


# Size in each dimension
# @markdown *Set size of the box to...*
size_x = 20.0 #@param {type:"raw"}
size_y = 20.0 #@param {type:"raw"}
size_z = 20.0 #@param {type:"raw"}

# @markdown *Options*
args = ""
allow_bad_res = True #@param {type:"boolean"}
if allow_bad_res:
  args += "--allow_bad_res "


# Prepare Receptor
# @markdown *Output*
prepare_outPDBQT = "5c45_receptorFH.pdbqt" #@param {type:"string"}
! python {mk_prepare_receptor} --macromol {prepare_inPDB} -o {prepare_outPDBQT} --box_center {center_x} {center_y} {center_z} --box_size {size_x} {size_y} {size_z} {args}


# Visualization with py3Dmol
def Receptor3DView(receptorPDB = None, boxPDB = None, ligPDB = None):

    view = py3Dmol.view()
    view.setBackgroundColor('white')

    view.addModel(open(boxPDB, 'r').read(),'pdb')
    view.addStyle({'stick': {}})
    view.zoomTo()

    view.addModel(open(receptorPDB, 'r').read(),'pdb')
    view.addStyle({'cartoon': {'color':'spectrum', 'opacity': 0.5}})

    if ligPDB is not None:
      view.addModel(open(ligPDB, 'r').read(), 'pdb')
      view.addStyle({'hetflag': True}, {'stick': {}})

    return view

Receptor3DView(receptorPDB = prepare_inPDB, \
               boxPDB = prepare_outPDBQT.replace('.pdbqt', '.box.pdb'), \
               ligPDB = prody_ligandPDB).show()

curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  223k    0  223k    0     0  1078k      0 --:--:-- --:--:-- --:--:-- 1079k


@> PDB file is found in working directory (5c45.pdb).
@> 2375 atoms and 1 coordinate set(s) were parsed in 0.05s.


Starting /usr/local/lib/python3.10/site-packages/mmtbx/command_line/reduce2.py
on Sat Sep 21 18:14:34 2024 by root

Processing files:
-------------------------------------------------------------------------------

  Found model, 5c45_receptor.pdb

Processing PHIL parameters:
-------------------------------------------------------------------------------

  Adding command-line PHIL:
  -------------------------
    approach=add
    add_flip_movers=True

Final processed PHIL parameters:
-------------------------------------------------------------------------------
  data_manager {
    model {
      file = "5c45_receptor.pdb"
    }
    default_model = "5c45_receptor.pdb"
  }
  add_flip_movers = True


Starting job
Writing model output to 5c45_receptorFH.pdb

                       ----------Loading Model----------                       


                      ----------Adding Hydrogens----------                     

Number of hydrogen atoms added to the input model: 1173 


           

@> PDB file is found in working directory (5c45.pdb).
@> 2375 atoms and 1 coordinate set(s) were parsed in 0.05s.


templates=<meeko.linked_rdkit_chorizo.ResidueChemTemplates object at 0x7c4e8136edd0>
@> 3513 atoms and 1 coordinate set(s) were parsed in 0.03s.
No template matched for residue_key='X:54'
tried 2 templates for residue_key='X:54'excess_H_ok=False
U          heavy_miss=16 heavy_excess=0 H_excess=0 bond_miss={11} bond_excess=set()
U-5endPO4  heavy_miss=17 heavy_excess=0 H_excess=0 bond_miss={13} bond_excess={1}

[18:14:49] product atom-mapping number 11 not found in reactants.
[18:14:49] product atom-mapping number 12 not found in reactants.
[18:14:49] product atom-mapping number 13 not found in reactants.
[18:14:49] product atom-mapping number 14 not found in reactants.
[18:14:49] product atom-mapping number 15 not found in reactants.
[18:14:49] product atom-mapping number 11 not found in reactants.
[18:14:49] product atom-mapping number 12 not found in reactants.

Files written:
     5c45_receptorFH.pdbqt <-- static (i.e., rigid) receptor input file
boron-silicon-atom_par.dat <-- atomic

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

CPU times: user 419 ms, sys: 21.6 ms, total: 440 ms
Wall time: 18.2 s


In [6]:
# @title # 2.3 [Basic Docking] Docking with Vina Scoring Function (~ 3min)
%%time

#@markdown *Input*
receptorPDBQT = "5c45_receptorFH.pdbqt" #@param {type:"string"}
ligandPDBQT = "51B.pdbqt" #@param {type:"string"}
configTXT = "5c45_receptorFH.box.txt" #@param {type:"string"}
#@markdown *Options*
exhaustiveness = 8 #@param {type:"raw"}
#@markdown *Output*
outputPDBQT = "5c45_51B_vina_out.pdbqt" #@param {type:"string"}

! vina --receptor {receptorPDBQT} --ligand {ligandPDBQT} --config {configTXT} \
       --exhaustiveness {exhaustiveness} \
       --out {outputPDBQT}

AutoDock Vina 36dd023-mod
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoD

In [7]:
# @title # 2.4 [Basic Docking] Export and Visualize Docked Poses (~ 1s)
%%time

# Export Docked Poses
#@markdown *Export docked poses to...*
dock_outSDF = "5c45_51B_vina_out.sdf" #@param {type:"string"}
! python {mk_export} {outputPDBQT} -o {dock_outSDF}

#@markdown *Visualize docked poses with...*
# Previously Generated Receptor Files
receptorPDB = "5c45_receptorFH.pdb" #@param {type:"string"}
boxPDB = "5c45_receptorFH.box.pdb" #@param {type:"string"}
refligPDB = 'LIG.pdb' #@param {type:"string"}
reflig_resn = '51B' #@param {type:"string"}

# Visualize Docked Poses
def Complex3DView(view, ligmol = None, refligPDB = None, reflig_resn = None):

    new_viewer = copy.deepcopy(view)

    mblock = Chem.MolToMolBlock(ligmol)
    new_viewer.addModel(mblock, 'mol')
    new_viewer.addStyle({'hetflag': True}, {"stick": {'colorscheme': 'greenCarbon'}})

    if refligPDB is not None and reflig_resn is not None:
      new_viewer.addModel(open(refligPDB, 'r').read(), 'pdb')
      new_viewer.addStyle({'resn': reflig_resn}, {"stick": {'colorscheme': 'magentaCarbon', 'opacity': 0.8}})

    return new_viewer


confs = Chem.SDMolSupplier(dock_outSDF)

def conf_viewer(idx):
    mol = confs[idx]
    return Complex3DView(Receptor3DView(receptorPDB = receptorPDB, boxPDB = boxPDB), \
                         mol, \
                         refligPDB = refligPDB, reflig_resn = reflig_resn).show()


interact(conf_viewer, idx=ipywidgets.IntSlider(min=0, max=len(confs)-1, step=1))

interactive(children=(IntSlider(value=0, description='idx', max=8), Output()), _dom_classes=('widget-interact'…

CPU times: user 66.8 ms, sys: 11.9 ms, total: 78.6 ms
Wall time: 2.69 s


<function __main__.conf_viewer(idx)>

In [8]:
# Download Zipped Output
! mkdir output; cp * output
! zip -r output.zip output

cp: -r not specified; omitting directory 'geostd'
cp: -r not specified; omitting directory 'Meeko'
cp: -r not specified; omitting directory 'output'
cp: -r not specified; omitting directory 'sample_data'
cp: -r not specified; omitting directory 'scrubber'
  adding: output/ (stored 0%)
  adding: output/5c45_receptor.pdb (deflated 75%)
  adding: output/vina (deflated 63%)
  adding: output/5c45_receptorFH.pdbqt (deflated 77%)
  adding: output/51B_scrubbed.sdf (deflated 77%)
  adding: output/condacolab_install.log (deflated 75%)
  adding: output/5c45_receptor_atoms.pdb (deflated 75%)
  adding: output/5c45_receptorFH.box.pdb (deflated 79%)
  adding: output/5c45_51B_vina_out.pdbqt (deflated 84%)
  adding: output/boron-silicon-atom_par.dat (deflated 39%)
  adding: output/5c45_51B_vina_out.sdf (deflated 84%)
  adding: output/51B.pdbqt (deflated 71%)
  adding: output/5c45_receptorFH.box.txt (deflated 40%)
  adding: output/5c45_receptorFH.pdb (deflated 75%)
  adding: output/5c45.pdb (deflated 77